# Amazon Bedrock + Langchain Sample Solution


## **What is LangChain?**
[LangChain](https://blog.langchain.dev/announcing-our-10m-seed-round-led-by-benchmark/#:~:text=LangChain%20is%20a%20framework%20for%20developing%20applications%20powered%20by%20language%20models) is a framework for developing applications powered by language models.It helps do this in two ways:
1. **Integration** - Bring external data, such as your files, other applications, and api data, to your LLMs
2. **Agency** - Allow your LLMs to interact with its environment via decision making. Use LLMs to help decide which action to take next

## **Why LangChain?**
1. **Components** - LangChain makes it easy to swap out abstractions and components necessary to work with language models.
2. **Customized Chains** - LangChain provides out of the box support for using and customizing 'chains' - a series of actions strung together.
3. **Speed 🚢** - This team ships insanely fast. You'll be up to date with the latest LLM features.
4. **Community 👥** - Wonderful [discord](https://discord.gg/6adMQxSpJS) and community support, meet ups, hackathons, etc.

Though LLMs can be straightforward (text-in, text-out) you'll quickly run into friction points that LangChain helps with once you develop more complicated applications.

## **Main Use Cases**

* **Text Generation**
* **Summarization** - One of the most common use case with LLM
* **Question and Answering Over Documents** - Use information held within documents to answer questions or query
* **Extraction** - Pull structured data from a body of text or an user query
* **Evaluation** - Understand the quality of output from your application
* **Querying Tabular Data** - Pull data from databases or other tabular source
* **Code Understanding** - Reason about and digest code
* **Chatbots** - A framework to have a back and forth interaction with a user combined with memory in a chat interface
* **Interacting with APIs** - Query APIs and interact with the outside world
* **Agents** - Use LLMs to make decisions about what to do next. Enable these decisions with tools.

## **Set up**

Install the Boto3 version that support BedRock.

In [ ]:
%pip uninstall -y -r requirements.txt

In [ ]:
%pip install -r requirements.txt

Set value for endpoint, AWS CLI profile, region & model name. 

In [ ]:
credentials_profile_name = "" # your AWS config profile
region = "" # for example, "us-east-1" or "us-west-2"
# model_id = "amazon.titan-tg1-large"
# model_id = "ai21.j2-mid"
model_id = "ai21.j2-ultra"
# model_id = "anthropic.claude-v2"
# model_id = "anthropic.claude-v1"
# model_id = "anthropic.claude-instant-v1"

Test your connection to BedRock

In [ ]:
import boto3
session = boto3.Session(profile_name=credentials_profile_name)
bedrock = session.client(
    service_name='bedrock', #creates a Bedrock client
    region_name=region
)
output_text = bedrock.list_foundation_models()
print(output_text) 

# Use Cases

## Text Generation

We will touch very briefly on this since it can also be done easily out of the box with BedRock playground or SDK

In [ ]:
from langchain.llms import Bedrock
llm = Bedrock(
    region_name = region,
    model_id=model_id,
    credentials_profile_name=credentials_profile_name
)
llm("Write me a Haiku")

## Summarization

One of the most common use cases for LangChain and LLMs is text summarization. Applications include Articles Summarization, Transcripts, Chat History, Slack/Discord, Customer Interactions, Medical Papers, Legal Documents, Podcasts, Tweet Threads, Code Bases, Product Reviews, Financial Documents

### Summaries Of Short Text

For summaries of short texts, the method is straightforward, in fact you don't need to do anything fancy other than simple prompting with instructions

In [ ]:
from langchain.llms import Bedrock
from langchain import PromptTemplate

llm = Bedrock(
    region_name = region,
    model_id=model_id,
    credentials_profile_name=credentials_profile_name
)

# Create our template
template = """
%INSTRUCTIONS:
Please summarize the following piece of text.
Respond in a manner that a 5 year old would understand.

%TEXT:
{text}
"""

# Create a LangChain prompt template that we can insert values to later
prompt = PromptTemplate(
    input_variables=["text"],
    template=template,
)

Let's let's find a confusing text online. *[Source](https://www.smithsonianmag.com/smart-news/long-before-trees-overtook-the-land-earth-was-covered-by-giant-mushrooms-13709647/)*

In [ ]:
confusing_text = """
For the next 130 years, debate raged.
Some scientists called Prototaxites a lichen, others a fungus, and still others clung to the notion that it was some kind of tree.
“The problem is that when you look up close at the anatomy, it’s evocative of a lot of different things, but it’s diagnostic of nothing,” says Boyce, an associate professor in geophysical sciences and the Committee on Evolutionary Biology.
“And it’s so damn big that when whenever someone says it’s something, everyone else’s hackles get up: ‘How could you have a lichen 20 feet tall?’”
"""

Let's take a look at what prompt will be sent to the LLM

In [ ]:
print ("------- Prompt Begin -------")

final_prompt = prompt.format(text=confusing_text)
print(final_prompt)

print ("------- Prompt End -------")

Finally let's pass it through the LLM

In [ ]:
output = llm(final_prompt)
print (output)

This method works fine, but for longer text, it can become a pain to manage and you'll run into token limits. Luckily LangChain has out of the box support for different methods to summarize via their [load_summarize_chain](https://python.langchain.com/en/latest/use_cases/summarization.html).

### Summaries Of Longer Text

*Note: This method will also work for short text too*

In [ ]:
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import Bedrock
llm = Bedrock(
    region_name = region,
    model_id=model_id,
    credentials_profile_name=credentials_profile_name
)

Let's load up a longer document

In [ ]:
with open('knowledge-repo/2022-share-holder-letter.txt', 'r') as file:
    text = file.read()

# Printing the first 285 characters as a preview
print (text[:285])

Then let's check how many tokens are in this document. [get_num_tokens](https://python.langchain.com/en/latest/reference/modules/llms.html#langchain.llms.OpenAI.get_num_tokens) is a nice method for this.

In [ ]:
num_tokens = llm.get_num_tokens(text)

print (f"There are {num_tokens} tokens in your file")

While you could likely stuff this text in your prompt, let's act like it's too big and needs another method.

First we'll need to split it up. This process is called 'chunking' or 'splitting' your text into smaller pieces. I like the [RecursiveCharacterTextSplitter](https://python.langchain.com/en/latest/modules/indexes/text_splitters/examples/recursive_text_splitter.html) because it's easy to control but there are a [bunch](https://python.langchain.com/en/latest/modules/indexes/text_splitters.html) you can try

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n"], chunk_size=5000, chunk_overlap=350)
docs = text_splitter.create_documents([text])

print (f"You now have {len(docs)} docs intead of 1 piece of text")

Next we need to load up a chain which will make successive calls to the LLM for us. Want to see the prompt being used in the chain below? Check out the [LangChain documentation](https://github.com/hwchase17/langchain/blob/master/langchain/chains/summarize/map_reduce_prompt.py)

For information on the difference between chain types, check out this video on [token limit workarounds](https://youtu.be/f9_BWhCI4Zo)

*Note: You could also get fancy and make the first 4 calls of the map_reduce run in parallel too*

In [ ]:
# Get your chain ready to use
chain = load_summarize_chain(llm=llm, chain_type='map_reduce') # verbose=True optional to see what is getting sent to the LLM

In [ ]:
# Use it. This will run through the documents, summarize the chunks, then get a summary of the summary.
output = chain.run(docs)
print (output)

## Question & Answering Using Documents As Context

In order to use LLMs for question and answer we must:

1. Pass the LLM relevant context it needs to answer a question
2. Pass it our question that we want answered

Simplified, this process looks like this "llm(your context + your question) = your answer"



### Simple Q&A Example

Here let's review the convention of `llm(your context + your question) = your answer`

In [ ]:
from langchain.llms import Bedrock
model_kwargs = {"temperature": 1}

llm = Bedrock(
    region_name = region,
    model_id=model_id,
    credentials_profile_name=credentials_profile_name
)

In [ ]:
context = """
Rachel is 30 years old
Bob is 45 years old
Kevin is 65 years old
"""

question = "Who is under 40 years old?"

Then combine them.

In [ ]:
output = llm(context + question)

# I strip the text to remove the leading and trailing whitespace
print (output.strip())

As we ramp up our sophistication, we'll take advantage of this convention more.

The hard part comes in when you need to be selective about *which* data you put in your context. This field of study is called "[document retrieval](https://python.langchain.com/en/latest/modules/indexes/retrievers.html)" and tightly coupled with AI Memory.

### Using RAG & Embeddings

In [ ]:
from langchain.llms import Bedrock

# The vectorstore we'll be using
from langchain.vectorstores import FAISS

# The LangChain component we'll use to get the documents
from langchain.chains import RetrievalQA

# The easy document loader for text
from langchain.document_loaders import TextLoader

# The embedding engine that will convert our text to vectors
from langchain.embeddings import BedrockEmbeddings

model_kwargs = {"temperature": 0}

llm = Bedrock(
    region_name = region,
    model_id=model_id,
    credentials_profile_name=credentials_profile_name
)

Let's load up a longer document

In [ ]:
loader = TextLoader('knowledge-repo/2022-share-holder-letter.txt')
doc = loader.load()
print (f"You have {len(doc)} document")
print (f"You have {len(doc[0].page_content)} characters in that document")

Now let's split our long doc into smaller pieces

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=400)
docs = text_splitter.split_documents(doc)

In [ ]:
# Get the total number of characters so we can see the average later
num_total_characters = sum([len(x.page_content) for x in docs])

print (f"Now you have {len(docs)} documents that have an average of {num_total_characters / len(docs):,.0f} characters (smaller pieces)")

In [ ]:
# Get your embeddings engine ready
embeddings = BedrockEmbeddings(region_name=region, credentials_profile_name=credentials_profile_name)

# Embed your documents and combine with the raw text in a pseudo db. Note: This will make an API call to BedRock Amazon Titan embedding
docsearch = FAISS.from_documents(docs, embeddings)

Create your retrieval engine

In [ ]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever())

Now it's time to ask a question. The retriever will go get the similar documents and combine with your question for the LLM to reason through.

Note: It may not seem like much, but the magic here is that we didn't have to pass in our full original document.

In [ ]:
query = "What is the plan for Amazon?"
qa.run(query)

If you wanted to do more you would hook this up to a cloud vector database, use a tool like metal and start managing your documents, with external data sources

## Extraction
*[LangChain Extraction Docs](https://python.langchain.com/en/latest/use_cases/extraction.html)*

Extraction is the process of parsing data from a piece of text. This is commonly used with output parsing in order to *structure* our data.

A popular library for extraction is [Kor](https://eyurtsev.github.io/kor/). We won't cover it today but I highly suggest checking it out for advanced extraction.

In [ ]:
# To help construct our Chat Messages
from langchain.schema import HumanMessage
from langchain.llms import Bedrock

model_kwargs = {"temperature": 0}

chat_model = Bedrock(
    region_name = region,
    model_id=model_id,
    credentials_profile_name=credentials_profile_name,
    model_kwargs = model_kwargs
)

In [ ]:
instructions = """
You will be given a sentence with fruit names, extract those fruit names and assign an emoji to them
Return the fruit name and emojis in a python dictionary
"""

fruit_names = """
Apple, Pear, this is an kiwi
"""

In [ ]:
# Make your prompt which combines the instructions w/ the fruit names
prompt = (instructions + fruit_names)

# Call the LLM
output = chat_model.predict_messages([HumanMessage(content=prompt)])

print (output.content)
print (type(output.content))

Let's turn this into a proper python dictionary

In [ ]:
output.content = '{"apple":"🍎","pear":"🍐","kiwi":"🥝"}'

In [ ]:
output_dict = eval(output.content)

print (output_dict)
print (type(output_dict))

## Evaluation

*[LangChain Evaluation Docs](https://python.langchain.com/en/latest/use_cases/evaluation.html)*

Evaluation is the process of doing quality checks on the output of your applications. Normal, deterministic, code has tests we can run, but judging the output of LLMs is more difficult because of the unpredictableness and variability of natural language. LangChain provides tools that aid us in this journey.


In [ ]:
# Embeddings, store, and retrieval
from langchain.embeddings import BedrockEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter


# Model and doc loader
from langchain.llms import Bedrock
from langchain.document_loaders import TextLoader

# Eval!
from langchain.evaluation.qa import QAEvalChain

model_kwargs = {"temperature": 0}
llm = Bedrock(
    region_name = region,
    model_id=model_id,
    credentials_profile_name=credentials_profile_name,
    model_kwargs = model_kwargs
)

In [ ]:
# Our long essay from before
loader = TextLoader('knowledge-repo/2022-share-holder-letter.txt')
doc = loader.load()

print (f"You have {len(doc)} document")
print (f"You have {len(doc[0].page_content)} characters in that document")

First let's do the Vectorestore dance so we can do question and answers

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=400)
docs = text_splitter.split_documents(doc)

# Get the total number of characters so we can see the average later
num_total_characters = sum([len(x.page_content) for x in docs])

print (f"Now you have {len(docs)} documents that have an average of {num_total_characters / len(docs):,.0f} characters (smaller pieces)")

In [ ]:
# Embeddings and docstore
embeddings = BedrockEmbeddings(region_name = region, credentials_profile_name=credentials_profile_name)
docsearch = FAISS.from_documents(docs, embeddings)

Make your retrieval chain. Notice how I have an `input_key` parameter now. This tells the chain which key from a dictionary I supply has my prompt/query in it. I specify `question` to match the question in the dict below

In [ ]:
chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever(), input_key="question")

Now I'll pass a list of questions and ground truth answers to the LLM that I know are correct (I validated them as a human).

In [ ]:
question_answers = [
    {'question' : "Which company sold the microcomputer kit that his friend built himself?", 'answer' : 'Healthkit'},
    {'question' : "What was the small city he talked about in the city that is the financial capital of USA?", 'answer' : 'Yorkville, NY'}
]

I'll use `chain.apply` to run both my questions one by one separately.

One of the cool parts is that I'll get my list of question and answers dictionaries back, but there'll be another key in the dictionary `result` which will be the output from the LLM.

Note: I specifically made my 2nd question ambigious and tough to answer in one pass so the LLM would get it incorrect

In [ ]:
predictions = chain.apply(question_answers)
predictions

We then have the LLM compare my ground truth answer (the `answer` key) with the result from the LLM (`result` key).

Or simply, we are asking the LLM to grade itself. What a wild world we live in.

In [ ]:
# Start your eval chain
eval_chain = QAEvalChain.from_llm(llm)

# Have it grade itself. The code below helps the eval_chain know where the different parts are
graded_outputs = eval_chain.evaluate(question_answers,
                                     predictions,
                                     question_key="question",
                                     prediction_key="result",
                                     answer_key='answer')

In [ ]:
graded_outputs

This is correct! Notice how the answer in question #1 was "Healthkit" and the prediction was "The microcomputer kit was sold by Heathkit." The LLM knew that the answer and result were the same and gave us a "correct" label. Awesome.

For #2 it knew they were not the same and gave us an "incorrect" label

## Querying Tabular Data

*[LangChain Querying Tabular Data Docs](https://python.langchain.com/en/latest/use_cases/tabular.html)*

The most common type of data in the world sits in tabular form (ok, ok, besides unstructured data). It is super powerful to be able to query this data with LangChain and pass it through to an LLM 

For futher reading check out "Agents + Tabular Data" ([Pandas](https://python.langchain.com/en/latest/modules/agents/toolkits/examples/pandas.html), [SQL](https://python.langchain.com/en/latest/modules/agents/toolkits/examples/sql_database.html), [CSV](https://python.langchain.com/en/latest/modules/agents/toolkits/examples/csv.html))

Let's query an SQLite DB with natural language. We'll look at the [San Francisco Trees](https://data.sfgov.org/City-Infrastructure/Street-Tree-List/tkzw-k3nq) dataset.

In [ ]:
from langchain.llms import Bedrock

from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

model_kwargs = {"temperature": 0}
llm = Bedrock(
    region_name = region,
    model_id=model_id,
    credentials_profile_name=credentials_profile_name,
    model_kwargs = model_kwargs
)

We'll start off by specifying where our data is and get the connection ready

In [ ]:
sqlite_db_path = 'knowledge-repo/San_Francisco_Trees.db'
db = SQLDatabase.from_uri(f"sqlite:///{sqlite_db_path}")

Then we'll create a chain that take our LLM, and DB. I'm setting `verbose=True` so you can see what is happening underneath the hood.

In [ ]:
db_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True)

In [ ]:
db_chain.run("How many distinct species of trees are there in San Francisco?")

This is awesome! There are actually a few steps going on here.

**Steps:**
1. Find which table to use
2. Find which column to use
3. Construct the correct sql query
4. Execute that query
5. Get the result
6. Return a natural language reponse back

Let's confirm via pandas

In [ ]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
connection = sqlite3.connect(sqlite_db_path)

# Define your SQL query
query = "SELECT count(distinct qSpecies) FROM SFTrees"

# Read the SQL query into a Pandas DataFrame
df = pd.read_sql_query(query, connection)

# Close the connection
connection.close()

In [ ]:
# Display the result in the first column first cell
print(df.iloc[0,0])

Nice! The answers match.

## Code Understanding

*[LangChain Code Understanding Docs](https://python.langchain.com/en/latest/use_cases/code.html)*

One of the most exciting abilities of LLMs is code undestanding. People around the world are leveling up their output in both speed & quality due to AI help. A big part of this is having a LLM that can understand code and help you with a particular task.


In [ ]:
# Helper to read local files
import os

# Vector Support
from langchain.vectorstores import FAISS
from langchain.embeddings import BedrockEmbeddings

# Model and chain
from langchain.llms.bedrock import Bedrock

# Text splitters
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader

llm = Bedrock(
    region_name = region,
    model_id=model_id,
    credentials_profile_name=credentials_profile_name
)

In [ ]:
embeddings = BedrockEmbeddings(region_name=region,credentials_profile_name=credentials_profile_name)

I put a small python package [The Fuzz](https://github.com/seatgeek/thefuzz) (personal indie favorite) in the data folder of this repo.

The loop below will go through each file in the library and load it up as a doc

In [ ]:
root_dir = 'knowledge-repo/investmate-ai'
docs = []
text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n"], chunk_size=512, chunk_overlap=20)


# Go through each folder
for dirpath, dirnames, filenames in os.walk(root_dir):
    
    # Go through each file
    for file in filenames:
        try: 
            # Load up the file as a doc and split
            loader = TextLoader(os.path.join(dirpath, file), encoding='utf-8')
            docs.extend(loader.load_and_split(text_splitter=text_splitter))
        except Exception as e: 
            pass

Let's look at an example of a document. It's just code!

In [ ]:
print (f"You have {len(docs)} documents\n")
print ("------ Start Document ------")
print (docs[0].page_content[:300])

Embed and store them in a docstore. This will make an API call to Amazon Titan embedding model

In [ ]:
docsearch = FAISS.from_documents(docs, embeddings)

In [ ]:
# Get our retriever ready
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever())

In [ ]:
query = "What function do I use to create memory for a chat session?"
output = qa.run(query)

In [ ]:
print (output)

## Chatbots

*[LangChain Chatbot Docs](https://python.langchain.com/en/latest/use_cases/chatbots.html)*

Chatbots use many of the tools we've already looked at with the addition of an important topic: Memory. There are a ton of different [types of memory](https://python.langchain.com/en/latest/modules/memory/how_to_guides.html), tinker to see which is best for you.


In [ ]:
from langchain.llms import Bedrock
from langchain import LLMChain
from langchain.prompts.prompt import PromptTemplate

# Chat specific components
from langchain.memory import ConversationBufferMemory

For this use case I'm going to show you how to customize the context that is given to a chatbot.

You could pass instructions on how the bot should respond, but also any additional relevant information it needs.

In [ ]:
template = """
Instruction: You are a chatbot that is unhelpful.
Your goal is to not help the user but only make jokes.
Take what the user is saying and make a joke out of it

{chat_history}
Human: {human_input}
Chatbot:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input"], 
    template=template
)
memory = ConversationBufferMemory(memory_key="chat_history")
llm = Bedrock(
    region_name = region,
    model_id=model_id,
    credentials_profile_name=credentials_profile_name
)

In [ ]:
llm_chain = LLMChain(
    llm = llm, 
    prompt=prompt, 
    verbose=True, 
    memory=memory
)

In [ ]:
llm_chain.predict(human_input="Is an pear a fruit or vegetable?")

In [ ]:
llm_chain.predict(human_input="Instruction: What was one of the fruits I first asked you about?")

Notice how my 1st interaction was put into the prompt of my 2nd interaction. This is the memory piece at work.

There are many ways to structure a conversation, check out the different ways on the [docs](https://python.langchain.com/en/latest/use_cases/chatbots.html)

## Interacting with APIs

*[LangChain API Interaction Docs](https://python.langchain.com/en/latest/use_cases/apis.html)*

If the data or action you need is behind an API, you'll need your LLM to interact with APIs. This topic is closely related to Agents and Plugins, though we'll look at a simple use case for this section. For more information, check out [LangChain + plugins](https://python.langchain.com/en/latest/use_cases/agents/custom_agent_with_plugin_retrieval_using_plugnplai.html) documentation.

In [ ]:
from langchain.chains import APIChain
from langchain.llms import Bedrock

model_kwargs = {"temperature": 0}
llm = Bedrock(
    region_name = region,
    model_id=model_id,
    credentials_profile_name=credentials_profile_name
)

LangChain's APIChain has the ability to read API documentation and understand which endpoint it needs to call.

In this case I wrote (purposefully sloppy) API documentation to demonstrate how this works

In [ ]:
api_docs = """

BASE URL: https://restcountries.com/

API Documentation:

The API endpoint /v3.1/name/{name} Used to find informatin about a country. All URL parameters are listed below:
    - name: Name of country - Ex: italy, france
    
The API endpoint /v3.1/currency/{currency} Uesd to find information about a region. All URL parameters are listed below:
    - currency: 3 letter currency. Example: USD, COP
    
Woo! This is my documentation
"""

chain_new = APIChain.from_llm_and_api_docs(llm, api_docs, verbose=True)

Let's try to make an API call that is meant for the country endpoint

In [ ]:
chain_new.run('Can you tell me information about france?')

Let's try to make an API call that is meant for the currency endpoint

In [ ]:
chain_new.run('Can you tell me about the currency COP?')

In both cases the APIChain read the instructions and understood which API call it needed to make.

Once the response returned, it was parsed and then my question was answered. Awesome 🐒

## Agents

*[LangChain Agent Docs](https://python.langchain.com/en/latest/modules/agents.html)*

Agents are one of the hottest [🔥](https://media.tenor.com/IH7C6xNbkuoAAAAC/so-hot-right-now-trending.gif) topics in LLMs. Agents are the decision makers that can look a data, reason about what the next action should be, and execute that action for you via tools

Examples of advanced uses of agents appear in [BabyAGI](https://github.com/yoheinakajima/babyagi) and [AutoGPT](https://github.com/Significant-Gravitas/Auto-GPT)


In [ ]:
# Helpers
import os
import json

from langchain.llms import Bedrock

# Agent imports
from langchain.agents import load_tools
from langchain.agents import initialize_agent

# Tool imports
from langchain.agents import Tool
from langchain.utilities import TextRequestsWrapper

In [ ]:
model_kwargs = {"temperature": 0}
llm = Bedrock(
    region_name = region,
    model_id=model_id,
    credentials_profile_name=credentials_profile_name
)

Create a tool to give the LLM the ability to make web service request

In [ ]:
requests = TextRequestsWrapper()

Put both your tools in a toolkit

In [ ]:
toolkit = [
    Tool(
        name = "Requests",
        func=requests.get,
        description="Useful for when you to make a request to a URL"
    ),
]

Create your agent by giving it the tools, LLM and the type of agent that it should be

In [ ]:
agent = initialize_agent(toolkit, llm, agent="zero-shot-react-description", verbose=True, return_intermediate_steps=True)

Now let's ask a question that requires making call to a URL. Due to the prompt template being used by Langchain, you may want to try another BedRock model if the current one does not work for this step.

In [ ]:
response = agent({"input":"Tell me about the content on this webpage https://aws.amazon.com/blogs/apn/harnessing-generative-ai-and-semantic-search-to-revolutionize-enterprise-knowledge-management/"})
response['output']